In [ ]:
import numpy as np
import sys
# sys.path.insert(0, '/Users/pouyabashivan/Dropbox (MIT)/Codes/dldata')
import dldata.metrics.utils as utils
import dldata.stimulus_sets.hvm as hvm
import cPickle
import h5py
import sys
import os
from sklearn.cross_validation import KFold
from sklearn import cross_decomposition
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed([0])
from dldata.physiology.hongmajaj.mappings import CHANNEL_INFO
from sklearn.preprocessing import scale
%matplotlib inline
sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/Dimensionality')

import seaborn as sns
from scipy.misc import imread
import re 

from image_optimizer import *
import scipy.misc
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
np.random.seed(123)
import tensorflow.contrib.slim as slim

%matplotlib inline
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops       
import scipy.stats as stats

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size": 14, "ytick.major.size": 14})
sns.set_context("poster")


dataset_hvm = hvm.HvMWithDiscfade()
meta_hvm = dataset_hvm.meta
neural_fea = np.array(dataset_hvm.neuronal_features)


# Utility Functions

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.interpolate import griddata, interp2d


def reps_to_array(reps):
    """
    reps: dictionary containing the reps for each variation level  
    """
    max_reps = np.max([reps[i].shape[0] for i in reps.keys()], axis=0)
    hvm_neural = np.zeros((max_reps, 5760, reps['V0'].shape[2]))
    hvm_neural[...] = np.NaN

    c = 0
    for key in reps:
        shape = reps[key].shape
        hvm_neural[:shape[0], c:c+shape[1], :] = reps[key]
        c += shape[1]
    return hvm_neural


def concat_reps(rep_list):
    """
    """
    max_reps = np.max([r.shape[0] for r in rep_list])
    resized_reps = []
    for r in rep_list:
        tmp = np.zeros((max_reps, r.shape[1], r.shape[2]))
        tmp[...] = np.NaN
        tmp[:r.shape[0], :, :] = r
        resized_reps.append(tmp)
    return np.concatenate(resized_reps, axis=-1)

def fix_nan_reps(reps):
    """Some of the entries in neural reps might be nan. 
    Substitute those values by the average response of 
    corresponding neurons to all images over all valid reps.
    reps = [n_reps, n_samples, n_neurons]
    """
    if np.any(np.isnan(reps)):
        # find the indices of nan neurons
        nan_ind = np.isnan(reps)
        _, _, nan_neu_ind = np.nonzero(nan_ind)

        corrected_reps = reps
        for n in np.unique(nan_neu_ind):
            # create a mask of all nan values for a neuron
            mask = np.zeros(shape=nan_ind.shape, dtype=bool)
            mask[:, :, n] = True
            masked_nan_ind = nan_ind & mask

            # substitue all nan values of neuron by average neuron response
            av_neuron_act = np.nanmean(reps[:, :, n])
            corrected_reps[masked_nan_ind] = av_neuron_act
        return corrected_reps
    else:
        return reps

def project_reps(input_reps, W_mat):
    """Project each rep of neural data using the projection matrix
    input_reps = [n_reps, n_samples, n_neurons]"""
    input_reps = fix_nan_reps(input_reps)
    reps = []
    for rep in input_reps:
        reps.append(scale(rep))
    comp_reps = np.tensordot(reps, W_mat, axes=1)
    return comp_reps

def fit_reg(X, Y):
    """
    Fits a linear regression model to the data and returns regression model with score and predictions"""
    reg = LinearRegression()
    reg.fit(X, Y)
    preds = reg.predict(X)
    score = pearsonr(Y, preds)
    return reg, score, preds

def predict_outputs(features, weights):
    """
    Predict outputs given input features and weights."""
    model_pcs = np.matmul(features - weights['pca_b'], weights['pca_w'])
    preds = np.matmul(model_pcs, weights['pls_w']) + weights['pls_b'] 
    return preds


def resize_mat(mat, new_size):
    """
    Resize a matrix to the desired size. Input size is [num_channels, num_pixels, num_pixels]"""
    if mat.ndim==2:
        mat = np.expand_dims(mat, axis=0)
    num_ch, _, num_pix = np.array(mat).shape

    x = np.arange(0, num_pix)
    y = np.arange(0, num_pix)
    ratio = (new_size - 1.) / (num_pix - 1)

    x_new = np.arange(0, new_size)
    y_new = np.arange(0, new_size)

    output = []
    for i in range(num_ch):
        resized_rf_func = interp2d(x * ratio, y * ratio, mat[i], kind='cubic')
        tmp_out = resized_rf_func(x_new, y_new)
        output.append(tmp_out)

    return np.squeeze(output)    


In [ ]:
# Load Left Magneto data and ID lookup dictionary (corresponding ids from hvm 5760 images)
from scipy.io import loadmat

data = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/mag_left_v4.mat')
images_lookup = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/id_object_lookup.mat')['id_object_lookup']
# site_locations = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/kk_2_pb_sites.mat')

# Find matching image ids on HVM
ids = []
for r in images_lookup:
    image_num = int(np.nonzero(meta_hvm['id'] == r[1])[0])
    ids.append(image_num)


# Separable (Mask-Mix) Mapping

In [ ]:
# TF implementation of RF limited Regression

class SeparableMap(object):
    def __init__(self, graph=None, rf_init=None, num_neurons=65, batch_size=50, init_lr=0.01,
                ls=0.05, ld=0.1, tol=1e-2, max_epochs=10, map_type='linreg', init_rfs=None):
        self.ld = ld    # reg factor for depth conv
        self.ls = ls    # reg factor for spatial conv
        self.tol = tol
        self.batch_size = batch_size
        self.num_neurons = num_neurons
        self.lr = init_lr
        self._lr_ph = tf.placeholder(dtype=tf.float32)
        self.max_epochs = max_epochs
        self.opt = tf.train.AdamOptimizer(learning_rate=self._lr_ph)
        self.map_type = map_type
        self.init_rfs = init_rfs
        
        if graph is None:
            self.graph = tf.Graph().as_default()
        else:
            self.graph = graph
            
    def iterate_minibatches(self, inputs, targets=None, batchsize=128, shuffle=False):
        input_len = inputs.shape[0]

        if shuffle:
            indices = np.arange(input_len)
            np.random.shuffle(indices)
        for start_idx in range(0, input_len, batchsize):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batchsize]
            else:
                excerpt = slice(start_idx, start_idx + batchsize)
            if targets is None:
                yield inputs[excerpt]
            else:
                yield inputs[excerpt], targets[excerpt] 
        
    def make_separable_map(self):
        with tf.variable_scope('mapping'):
            if self.map_type == 'separable':
                input_shape = self.input_ph.shape
                preds = []
                for n in range(self.num_neurons):
                    with tf.variable_scope('N_{}'.format(n)):
                        if self.init_rfs is None:
                            s_w = tf.Variable(initial_value=np.random.randn(1, input_shape[1], input_shape[2], 1), dtype=tf.float32)
                        else:
                            assert self.init_rfs.shape == (self.num_neurons, input_shape[1], input_shape[2]), \
                            'Filter initialization matrix should be ({},{},{})'.format(self.num_neurons, input_shape[1], input_shape[2])
                            s_w = tf.Variable(initial_value=self.init_rfs[n].reshape((1, input_shape[1], input_shape[2], 1)), dtype=tf.float32)
                        tf.add_to_collection('s_w', s_w)
                        out = self.input_ph * s_w
                        d_w = tf.Variable(initial_value=np.random.randn(1, 1, out.shape[-1], 1), dtype=tf.float32)
                        tf.add_to_collection('d_w', d_w)
                        out = tf.nn.conv2d(out, d_w, [1, 1, 1, 1], 'SAME')
                        bias = tf.Variable(initial_value=np.zeros((1, 1, 1, 1)), dtype=tf.float32)
                        preds.append(tf.reduce_sum(out, axis=[1, 2]) + bias)
                self._predictions = tf.concat(preds, -1)
            elif self.map_type == 'linreg':
                # For L1-Regression
                tmp = tf.layers.flatten(self.input_ph)
                self._predictions = tf.layers.dense(tmp, self.num_neurons)
            
    def make_loss(self):    
        with tf.variable_scope('loss'):
            self.l2_error = tf.norm(self._predictions - self.target_ph, ord=2) #  tf.reduce_sum(tf.pow(self._predictions-self.target_ph, 2))/(2*self.batch_size) # 

            # For L1-Regression
            if self.map_type == 'linreg':
                self.reg_loss = tf.reduce_sum([tf.reduce_sum(tf.abs(t)) for t in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)])
                self.total_loss = self.l2_error + self.reg_loss
                
            elif self.map_type == 'separable':
                # For separable mapping
                self.s_vars = tf.get_collection('s_w')
                self.d_vars = tf.get_collection('d_w')

                # L1 reg
#                 self.reg_loss = self.ls * tf.reduce_sum([tf.reduce_sum(tf.abs(t)) for t in self.s_vars]) + self.ld * tf.reduce_sum([tf.reduce_sum(tf.abs(t)) for t in self.d_vars])
                # L2 reg
#                 self.reg_loss = self.ls * tf.reduce_sum([tf.reduce_sum(tf.pow(t, 2)) for t in self.s_vars]) + self.ld * tf.reduce_sum([tf.reduce_sum(tf.pow(t, 2)) for t in self.d_vars])
#                 self.total_loss = self.l2_error + self.reg_loss

                # Laplacian loss
                laplace_filter = tf.constant(np.array([0, -1, 0, -1, 4, -1, 0, -1, 0]).reshape((3, 3, 1, 1)), dtype=tf.float32)
                laplace_loss = tf.reduce_sum([tf.norm(tf.nn.conv2d(t, laplace_filter, [1, 1, 1, 1], 'SAME')) for t in self.s_vars])
                l2_loss = tf.reduce_sum([tf.reduce_sum(tf.pow(t, 2)) for t in self.s_vars])
                self.reg_loss = self.ls * (l2_loss + laplace_loss) + \
                                self.ld * tf.reduce_sum([tf.reduce_sum(tf.pow(t, 2)) for t in self.d_vars])

                self.total_loss = self.l2_error + self.reg_loss
            self.tvars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            self.train_op = self.opt.minimize(self.total_loss, var_list=self.tvars, 
                                             global_step=tf.contrib.framework.get_or_create_global_step())
    
    def fit(self, X, Y):
        if self.map_type == 'linreg':
            assert X.ndim == 2, 'Input matrix rank should be 2.'
        else:
            assert X.ndim == 4, 'Input matrix rank should be 4.'
        self.input_ph = tf.placeholder(dtype=tf.float32, shape=[None]+list(X.shape[1:]))
        self.target_ph = tf.placeholder(dtype=tf.float32, shape=[None, Y.shape[-1]])
        # Build the model graph
        self.model = self.make_separable_map()
        self.make_loss()
        
        # initialize graph
        print('Initializing...')
        self.sess = tf.Session()
        init_op = tf.variables_initializer(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
        self.sess.run(init_op)
        for e in range(self.max_epochs):
            for counter, batch in enumerate(self.iterate_minibatches(X, Y, batchsize=self.batch_size, shuffle=True)):
                feed_dict = {self.input_ph: batch[0], 
                             self.target_ph: batch[1],
                            self._lr_ph: self.lr}
                _, loss_value, reg_loss_value = self.sess.run([self.train_op, self.l2_error, self.reg_loss], feed_dict=feed_dict)
            if e % 100 == 0:
                print('Epoch: %d, Err Loss: %.2f, Reg Loss: %.2f' % (e+1, loss_value, reg_loss_value))
            if e % 200 == 0 and e != 0:
                self.lr /= 10.
            if loss_value < self.tol:
                print('Converged.')
                break
            
                
    def predict(self, X):
        preds = []
        for batch in self.iterate_minibatches(X, batchsize=self.batch_size, shuffle=False):
            feed_dict = {self.input_ph: batch}
            preds.append(np.squeeze(self.sess.run([self._predictions], feed_dict=feed_dict)))
        return np.concatenate(preds, axis=0)        


In [ ]:
from sklearn.decomposition import PCA
import h5py
from scipy.stats import pearsonr

map_type = 'separable'


# feats = np.array(h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_model_139k.hdf5')['mdl_conv3'])[ids]
# feats = feats.reshape(-1, 17, 17, 384)

# n_comps = 1000
# pca = PCA(n_components=n_comps)
# pca_feats = pca.fit_transform(feats)

# v4_data_reps = np.array(h5py.File('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/season5/hvm_initial_new_normalized_rates_kk_pb.mat')['normalized_rates'])
# v4_data_reps = v4_data_reps.transpose(2, 0, 1)
# neurons = np.nanmean(v4_data_reps, axis=0)

# Load RF data
v4_rf_cells = np.array(h5py.File('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/rf_mask_perCell.mat')['rf_mask_percell'])
v4_rf_cells = v4_rf_cells.transpose(2, 0, 1)
rf_inits = resize_mat(v4_rf_cells, 17)

if map_type == 'linreg':
    X = pca_feats
else:
    X = feats
Y = scale(neurons)
    
train_ind = np.random.choice(range(X.shape[0]), 576, replace=False)
test_ind = np.nonzero(~ np.in1d(range(X.shape[0]), train_ind))[0]

all_scores = np.zeros((3, 3))

for ls_ind, ls in enumerate([1, 2, 5]):  # [0.01, 0.05, 0.1, 0.2, 0.5]
    for ld_ind, ld in enumerate([1, 2, 5]):
        with tf.Graph().as_default() as graph:
            with tf.Session() as sess:
                print 'Mapping with ls={}, ld={}'.format(ls, ld)
                mapper = SeparableMap(graph=graph, max_epochs=1000, tol=0.1, 
                                      init_lr=0.01, batch_size=50, ls=ls, ld=ld, 
                                      map_type=map_type, init_rfs=rf_inits)
                mapper.fit(X[train_ind], Y[train_ind])
                preds = mapper.predict(X[test_ind])
                scores = [pearsonr(preds[:, i], Y[test_ind, i])[0] for i in range(preds.shape[-1])]
                all_scores[ls_ind, ld_ind] = np.median(scores)